In [420]:
import pandas as pd
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.use('pgf')
# 'pgf'
import numpy as np
from sklearn.metrics import mean_squared_error
from pathlib import Path
import os
import pickle
from models.gan.gan import generator, discriminator, StockTimeGan
from utils.log import prepare_logger
from pathlib import Path
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime

plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
    "figure.figsize": (30, 10), 
    "figure.dpi": 600
})


logger = prepare_logger("INFO")
# %load_ext blackcellmagic

# %%black for formatting


In [421]:
TICKER = "EA"
MODEL_VERSION = 0.6

In [422]:
load_path = Path(os.path.abspath("")).parents[0] / "data" / "scaled_data"
models_path = Path(os.path.abspath("")).parents[0] / "models" / "gan" / "versions"

with open(load_path / f"data_{TICKER}.pickle", "rb") as handle:
    data = pickle.load(handle)

with open(load_path / f"scalers_{TICKER}.pickle", "rb") as handle:
    scalers = pickle.load(handle)

with open(models_path / f"model_{MODEL_VERSION}_{TICKER}.pickle", "rb") as handle:
    model_data = pickle.load(handle)


In [423]:
test_preds = scalers["y_scaler"].inverse_transform(model_data["test_preds"])
test_true = scalers["y_scaler"].inverse_transform(model_data["actual_values"])


In [424]:
test_true

array([[134.86680603],
       [137.44718933],
       [141.0138855 ],
       [140.75485229],
       [141.4821167 ],
       [141.02384949],
       [139.93789673],
       [140.92422485],
       [141.58175659],
       [141.08361816],
       [142.23930359],
       [140.52571106],
       [139.11099243],
       [140.33641052],
       [140.51573181],
       [140.73490906],
       [142.60794067],
       [143.25550842],
       [142.92674255],
       [143.45478821],
       [145.79602051],
       [141.55187988],
       [140.6552124 ],
       [136.55053711],
       [137.34754944],
       [140.81460571],
       [141.89059448],
       [140.12718201],
       [140.83456421],
       [139.15083313],
       [138.612854  ],
       [138.10472107],
       [137.01878357],
       [137.54681396],
       [138.36376953],
       [141.28288269],
       [139.64898682],
       [141.56182861],
       [142.72747803],
       [143.45478821],
       [142.39871216],
       [142.39871216],
       [143.25653076],
       [141

In [425]:
test_true[0][0]

134.86680603027344

In [426]:
test_true[-1]

array([125.63999939])

In [427]:
amount = round(1000 / test_true[0][0])
rest = 1000 - 67 * test_true[0][0]
amount

7

In [428]:
67 * test_true[-1][0] + rest

381.803955078125

In [429]:
# plt.plot(data["Y_preds_real_list_train"])
# plt.show()

In [430]:
# plt.plot(data["Y_preds_real_list_test"])
# plt.show()

In [431]:
# full_y = list(scalers["y_scaler"].inverse_transform(data["Y_preds_real_list_train"]))
# full_y.extend((list(scalers["y_scaler"].inverse_transform(data["Y_preds_real_list_test"]))))

# plt.plot(full_y)
# plt.axvline(535, label='Train/Test boundary')
# plt.title('EA Close price')
# plt.legend()
# plt.show()

In [432]:
data["Y_preds_real_list_train"].shape

(535, 1)

In [433]:
data["X_list_train"].shape

(535, 30, 19)

In [434]:
data["Y_preds_real_list_test"].shape

(178, 1)

In [435]:
mean_absolute_error(y_true=test_true, y_pred=test_preds)


1.7336166896177148

In [436]:
np.sqrt(mean_squared_error(y_true=test_true, y_pred=test_preds))


2.323433961433916

In [437]:
df = pd.DataFrame(
    {
        "predictions": test_preds.reshape(
            test_preds.shape[0],
        ),
        "actual_values": test_true.reshape(
            test_true.shape[0],
        ),
    }
)


In [438]:
stock_path = Path(os.path.abspath("")).parents[0] / "data" / "stock_data" / f'{TICKER}.csv'
stock_df = pd.read_csv(stock_path)

In [439]:
stock_df["Date"] = pd.to_datetime(stock_df["Date"])
dates = (
    stock_df[
        stock_df["Date"]
        >= datetime.datetime(year=2019, month=1, day=1) + datetime.timedelta(days=30)
    ]
    .tail(178)["Date"]
    .values
)


In [440]:
pd.DataFrame({"dates": dates}).to_csv(
    Path(os.path.abspath("")).parents[0] / "data" / "test_dates.csv", index=False
)


In [441]:
fig_save_path = Path(os.path.abspath("")).parents[0] / "paper"

In [442]:
fig_save_path = Path(os.path.abspath("")).parents[0] / "paper"
plt.plot(
    dates,
    test_preds.reshape(
        test_preds.shape[0],
    ),
    color="blue",
    label="Predicted price",
)
plt.plot(
    dates,
    test_true.reshape(
        test_true.shape[0],
    ),
    color="red",
    label="Actual price",
)
plt.title(f"{TICKER} actual price vs predicted price", size=25)
plt.xlabel("Date", size=25)
plt.ylabel("Valuation", size=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=25)
plt.show()
plt.savefig(fig_save_path / f'{TICKER}_prices.pgf')
plt.clf()


C:\Users\kuba1\AppData\Local\Temp\ipykernel_19208\3207918460.py:24: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


<Figure size 18000x6000 with 0 Axes>

In [443]:
np.median(test_true)


140.66913604736328

In [444]:
np.std(test_true)


5.457249515119892

In [445]:
results = {"1000_epochs_no_sentiment": {"mea": 1.2186, "rmse": 1.7288}}

In [446]:
plt.plot(
    model_data['train_history']['gen_loss'],
    color="blue",
    label="Generator loss",
)
plt.plot(
    model_data['train_history']['disc_loss'],
    color="red",
    label="Discriminator Loss",
)
plt.title(f"{TICKER} generator and discriminator loss", size=25)
plt.xlabel("Epoch", size=25)
plt.ylabel("Loss", size=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=25)
plt.show()
plt.savefig(fig_save_path / f'{TICKER}_loss.pgf')
plt.clf()

C:\Users\kuba1\AppData\Local\Temp\ipykernel_19208\3519179210.py:17: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


<Figure size 18000x6000 with 0 Axes>

In [447]:
model_data['train_history'].keys()

dict_keys(['gen_loss', 'disc_loss', 'real_y', 'pred_y'])

O czym napisac: 
- Spotkanie 
- Problemy ze stabilizacja modelu 
- Wywalenie dense, dodawanie i wywalanie sentymentu 
- Co z innymi modelami 
- Co jeszcze potrzeba 
- Wykresy? 
- Czy inne firmy 